<a href="https://colab.research.google.com/github/p09323028/2021s_NTU_Big-Data-Marketing/blob/main/Homework/05_%E4%BD%9C%E6%A5%AD_%E5%95%A4%E9%85%92%E8%88%87%E5%B0%BF%E5%B8%83%E5%BB%9A%E5%85%B7%E8%88%87%E5%85%A7%E8%A4%B2%E8%B3%BC%E7%89%A9%E7%B1%83%E5%88%86%E6%9E%90_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Author: Jinze Wu

Student Number: p09323028

In [ ]:
!pip install factor_analyzer
import numpy as np
import pandas as pd
import datetime as dt
from factor_analyzer import FactorAnalyzer

# **Download data**
Download data from google cloud

In [ ]:
!gdown --id '1oTtu7K5NVLCp-RKUuyVtwShLnryIVrKc' --output creditcard.xlsx

# **Sheet to DataFrame**

In [ ]:
customer = pd.read_excel('creditcard.xlsx', sheet_name='客戶資料檔')
creditcard = pd.read_excel('creditcard.xlsx', sheet_name='信用卡資料檔')
creditlog = pd.read_excel('creditcard.xlsx', sheet_name='信用卡交易記錄檔')
creditlog = creditlog.iloc[:,:8]

In [ ]:
customer.head()

,客戶ID,年齡,生日,居住地,教育程度,性別,婚姻狀況,職業,辦第一張信用卡的時間
0,7293,57,1955-08-24,大台北地區,專科,1:男,已婚,公務員,2006-11-27
1,5963,29,1983-12-05,大台北地區,學士,1:男,單身,白領一般職員,2009-07-24
2,18700,27,1985-12-20,高屏地區,高中,1:男,單身,藍領(勞動工作或作業員)一般職員,2009-09-22
3,920,39,1973-05-29,大台北地區,學士,1:男,已婚,自營事業老闆,2010-03-05
4,4612,26,1986-01-06,嘉南地區,學士,1:男,單身,白領一般職員,2010-08-05


In [ ]:
creditcard.head()

,信用卡ID,客戶ID,信用卡開卡日,信用卡到期日,信用額度,卡等
0,41,2027,2006-05-19,2015-08-31,500000,金卡
1,814,7437,2002-01-14,2016-01-31,200000,普卡
2,905,6475,2002-05-07,2016-05-31,50000,普卡
3,1142,18841,2001-07-23,2015-07-31,250000,金卡
4,1411,14127,2005-10-03,2016-06-30,270000,白金卡


In [ ]:
creditlog.head()

,交易序號,信用卡ID,客戶ID,刷卡日期,刷卡產品產業分類,刷卡類型,刷卡金額,刷卡地點
0,8,6448,15195,2010-12-01,14_旅遊,一般消費,5200,國內
1,9,6826,13687,2010-12-01,12_量販超市,一般消費,250,國內
2,10,6826,13687,2010-12-01,12_量販超市,一般消費,1316,國內
3,11,7896,179,2010-12-01,10_藥妝,一般消費,218,國內
4,20,9387,16756,2010-12-01,11_3C居家電信,一般消費,2610,國內


# **Create BuyOrNot Table**

In [ ]:
creditlog.columns

Index(['交易序號', '信用卡ID', '客戶ID', '刷卡日期', '刷卡產品產業分類', '刷卡類型', '刷卡金額', '刷卡地點'], dtype='object')

In [ ]:
df = pd.get_dummies(creditlog[['客戶ID','刷卡產品產業分類']], columns=['刷卡產品產業分類'], prefix='', prefix_sep='').groupby('客戶ID').agg(lambda x: 1 if sum(x)>=1 else 0)

## **Correlation Matrix**

In [ ]:
df.corr()

,02_學費/教育,03_保險,05_捐贈,06_公用事業,07_百貨,08_精品,09_餐飲,10_藥妝,11_3C居家電信,12_量販超市,13_交通(含加值),14_旅遊,15_休閒文化,16_無店舖,X2.中信錢加值
02_學費/教育,1.000000,0.357190,0.164221,0.062803,-0.023667,0.003054,0.091579,0.128906,0.085585,0.112611,0.144370,0.103729,0.079635,0.185003,-0.051859
03_保險,0.357190,1.000000,0.076469,-0.063076,0.150629,0.149429,0.160552,0.272262,0.204619,0.121996,0.117579,0.393311,0.088934,0.191930,0.076469
05_捐贈,0.164221,0.076469,1.000000,0.076469,0.162314,-0.013356,0.069298,0.210964,0.065708,0.115129,0.131897,0.076591,0.179416,0.250585,-0.030928
06_公用事業,0.062803,-0.063076,0.076469,1.000000,0.097215,0.039149,0.000533,0.001624,0.044760,0.063903,0.117579,-0.034783,-0.017574,0.135313,-0.079590
07_百貨,-0.023667,0.150629,0.162314,0.097215,1.000000,0.291736,0.346533,0.320648,0.284338,0.183892,0.101994,0.229885,0.382904,0.263705,0.044695
08_精品,0.003054,0.149429,-0.013356,0.039149,0.291736,1.000000,0.315811,0.413966,0.258697,0.230510,0.272712,0.472538,0.367510,0.255043,-0.013356
09_餐飲,0.091579,0.160552,0.069298,0.000533,0.346533,0.315811,1.000000,0.396377,0.378585,0.179262,0.330596,0.275877,0.319439,0.233933,0.069298
10_藥妝,0.128906,0.272262,0.210964,0.001624,0.320648,0.413966,0.396377,1.000000,0.297900,0.102047,0.286344,0.267692,0.321718,0.366244,0.210964
11_3C居家電信,0.085585,0.204619,0.065708,0.044760,0.284338,0.258697,0.378585,0.297900,1.000000,0.148507,0.178476,0.177028,0.259460,0.304787,0.065708
12_量販超市,0.112611,0.121996,0.115129,0.063903,0.183892,0.230510,0.179262,0.102047,0.148507,1.000000,0.145479,0.109659,0.117861,0.088176,0.115129


# **Factor Analysis**

In [ ]:
fa = FactorAnalyzer(n_factors=5,rotation='varimax',method='principal')
results = fa.fit(df)

## **Communalities**

In [ ]:
pd.DataFrame({
    'Catagory': df.columns,
    'Communalities': results.get_communalities()
}).set_index('Catagory')

,Communalities
Catagory,
02_學費/教育,0.700719
03_保險,0.709357
05_捐贈,0.503677
06_公用事業,0.581312
07_百貨,0.436155
08_精品,0.620651
09_餐飲,0.446260
10_藥妝,0.516817
11_3C居家電信,0.335748


## **Rotated Component Matrix**

In [ ]:
RCM = pd.DataFrame({
    'Factor1': results.loadings_.T[0],
    'Factor2': results.loadings_.T[1],
    'Factor3': results.loadings_.T[2],
    'Factor4': results.loadings_.T[3],
    'Factor5': results.loadings_.T[4]},
    index=df.columns)

RCM = RCM.sort_values(by=['Factor1'], ascending=False)

In [ ]:
RCM = RCM[RCM>0.4].fillna('')

In [ ]:
index_order = ['08_精品','15_休閒文化','14_旅遊','09_餐飲','07_百貨','10_藥妝','11_3C居家電信','05_捐贈','16_無店舖','13_交通(含加值)','03_保險','02_學費/教育','X2.中信錢加值','12_量販超市','06_公用事業']
RCM.reindex(index_order)

,Factor1,Factor2,Factor3,Factor4,Factor5
08_精品,0.77206,,,,
15_休閒文化,0.642052,,,,
14_旅遊,0.639154,,,,
09_餐飲,0.620414,,,,
07_百貨,0.599641,,,,
10_藥妝,0.532038,,,,
11_3C居家電信,0.491139,,,,
05_捐贈,,0.694916,,,
16_無店舖,,0.669482,,,
13_交通(含加值),,0.404795,,,


## **Total Variance Explained**

### **Initial Eigenvalues**

In [ ]:
t = results.get_eigenvalues()[0]
vp = t / t.sum() * 100
vp_cum = vp.cumsum()

In [ ]:
IE = pd.DataFrame({
    'Total': t,
    '% of Var': vp,
    'Cummulative %': vp_cum},
    index=list(range(1,16)))
IE

,Total,% of Var,Cummulative %
1,3.787100,25.247331,25.247331
2,1.334980,8.899869,34.147201
3,1.259551,8.397004,42.544205
4,1.099421,7.329471,49.873676
5,1.016696,6.777975,56.651650
6,0.978765,6.525098,63.176748
7,0.926364,6.175761,69.352510
8,0.830540,5.536931,74.889441
9,0.723627,4.824179,79.713620
10,0.673412,4.489412,84.203033


### **Rotation Sums of Squared Loadings**

In [ ]:
RSSL = pd.DataFrame({
    'Total': np.array(results.get_factor_variance())[0],
    '% of Var': np.array(results.get_factor_variance())[1] * 100,
    'Cummulative %': np.array(results.get_factor_variance())[2] * 100},
    index=list(range(1,6)))
RSSL

,Total,% of Var,Cummulative %
1,3.062468,20.416451,20.416451
2,1.628598,10.857318,31.273769
3,1.503036,10.020237,41.294006
4,1.191995,7.946634,49.240640
5,1.111652,7.411011,56.651650


## **Component Transform Matrix**

In [ ]:
CTM = pd.DataFrame({
    '1': results.rotation_matrix_.T[0],
    '2': results.rotation_matrix_.T[2],
    '3': results.rotation_matrix_.T[1],
    '4': results.rotation_matrix_.T[3],
    '5': results.rotation_matrix_.T[4]},
    index=list(range(1,6)))
CTM

,1,2,3,4,5
1,0.843114,0.401025,0.289960,0.170555,-0.123171
2,-0.468168,0.481462,0.705221,-0.143823,-0.176044
3,-0.075153,0.596310,-0.619168,-0.281195,-0.419913
4,-0.253601,0.274329,-0.156056,0.912787,0.053815
5,0.003998,0.420156,-0.104229,-0.194872,0.880122


## **Factor Scores**

In [ ]:
factor_scores = pd.DataFrame(
            results.transform(df),
            columns = ['Factor1','Factor2','Factor3','Factor4','Factor5'],
            index = df.index)
factor_scores

,Factor1,Factor2,Factor3,Factor4,Factor5
客戶ID,,,,,
89,0.176096,-0.818682,-0.053605,-0.002069,0.368057
106,0.651081,-1.006710,0.263592,0.251618,-0.049569
131,0.043160,-0.251690,1.264664,-0.290305,-1.942042
179,0.149324,0.248497,-0.947260,-0.150177,-1.387779
261,-0.571216,0.855781,-0.560396,0.352732,-0.667150
...,...,...,...,...,...
19239,-0.557374,0.058584,0.142390,0.081997,-1.750708
19483,1.037991,3.210003,-0.321806,-0.348148,-0.251041
19504,1.823930,-0.645700,-0.342526,0.003220,0.063993


# **SPSS**

```
MATRIX DATA VARIABLES = A1 TO A15
/N = 1000
/CONTENTS = CORR .
BEGIN DATA.
1.0000 
0.3572  1.0000 
0.1642  0.0765  1.0000 
0.0628  -0.0631  0.0765  1.0000 
-0.0237  0.1506  0.1623  0.0972  1.0000 
0.0031  0.1494  -0.0134  0.0391  0.2917  1.0000 
0.0916  0.1606  0.0693  0.0005  0.3465  0.3158  1.0000 
0.1289  0.2723  0.2110  0.0016  0.3206  0.4140  0.3964  1.0000 
0.0856  0.2046  0.0657  0.0448  0.2843  0.2587  0.3786  0.2979  1.0000 
0.1126  0.1220  0.1151  0.0639  0.1839  0.2305  0.1793  0.1020  0.1485  1.0000 
0.1444  0.1176  0.1319  0.1176  0.1020  0.2727  0.3306  0.2863  0.1785  0.1455  1.0000 
0.1037  0.3933  0.0766  -0.0348  0.2299  0.4725  0.2759  0.2677  0.1770  0.1097  0.2596  1.0000 
0.0796  0.0889  0.1794  -0.0176  0.3829  0.3675  0.3194  0.3217  0.2595  0.1179  0.3184  0.4001  1.0000 
0.1850  0.1919  0.2506  0.1353  0.2637  0.2550  0.2339  0.3662  0.3048  0.0882  0.3491  0.1774  0.4182  1.0000 
-0.0519  0.0765  -0.0309  -0.0796  0.0447  -0.0134  0.0693  0.2110  0.0657  0.1151  0.1319  0.0766  0.0622  0.1259  1.0000

END DATA.
EXECUTE.
FACTOR
/MATRIX = IN (COR = *)
/PRINT INITIAL EXTRACTION ROTATION
/FORMAT SORT BLANK(.4)
/CRITERIA MINEIGEN(1) ITERATE(40) /EXTRACTION PC
/CRITERIA ITERATE(25) /ROTATION VARIMAX
/CRITERIA ITERATE(25) /ROTATION VARIMAX
/METHOD=CORRELATION.
```

## **Output file**
https://tinyurl.com/2f4k7ra6